In [4]:
import os

In [5]:
%pwd

'/config/workspace/End-to-end-ML-project/research'

In [6]:
os.chdir("../")

In [7]:
%pwd

'/config/workspace/End-to-end-ML-project'

In [8]:
# Create entity: entity is nothing but a return type of variable
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [9]:
# Import red_wine_quality_project\constants
from red_wine_quality_project.constants import *
# from src.red_wine_quality_project.constants import *

# Import utils
from red_wine_quality_project.utils.common import read_yaml, create_directories

In [10]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [11]:
# update the components
import os
import urllib.request as request
import zipfile
from red_wine_quality_project import logger
from red_wine_quality_project.utils.common import get_size

In [12]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    # step 1 : download zip file
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    # step 2 : extract zip file
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [14]:
# update the pipeline 

try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-01-07 11:41:40,609: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-01-07 11:41:40,611: INFO: common: yaml file: params.yaml loaded successfully]
[2024-01-07 11:41:40,615: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-01-07 11:41:40,617: INFO: common: created directory at: artifacts]
[2024-01-07 11:41:40,618: INFO: common: created directory at: artifacts/data_ingestion]
[2024-01-07 11:41:41,058: INFO: 813279425: artifacts/data_ingestion/data.zip download! with following info: 
Server: GitHub.com
Date: Sun, 07 Jan 2024 06:11:40 GMT
Content-Type: text/html; charset=utf-8
Vary: X-PJAX, X-PJAX-Container, Turbo-Visit, Turbo-Frame, Accept-Encoding, Accept, X-Requested-With
ETag: W/"5d7359d1ca166e57e4cfe8d0ce243323"
Cache-Control: max-age=0, private, must-revalidate
Strict-Transport-Security: max-age=31536000; includeSubdomains; preload
X-Frame-Options: deny
X-Content-Type-Options: nosniff
X-XSS-Protection: 0
Referrer-Policy: no-referrer-when-dow

BadZipFile: File is not a zip file